#### Minimizing an FSA

Consider  `A = (T, Q, R, q₀, F)` with `T = {a, b}`, `Q = {0, 1, 2, 3}`, `q₀ = 0`, `F = {2}`, and transitions `R`:  
 
    0 a → 1
    0 b → 0
    1 a → 1  
    1 b → 2  
    2 a → 3  
    2 b → 2  
    3 a → 3  
    3 b → 2  

Minimize `A` following the method from the course notes by filling the table below. Add `⓪` for states that are initially identified as distict, `①` for states that are identifed as distict in the first round, `②` for states that are identified as distinct in the second round, etc. Add explanations! 

|     | `0` | `1` | `2` | `3` |
|:---:|:---:|:---:|:---:|:---:|
| `0` |     |     |     |     |
| `1` |     |     |     |     |
| `2` |     |     |     |     |
| `3` |     |     |     |     |

Construct the new states, the new initial state, the new final states, and the transitions. You may check your result by running `minimizeFSM` further below.

Initial: final state vs. other
|     | `0` | `1` | `2` | `3` |
|:---:|:---:|:---:|:---:|:---:|
| `0` |     |     |  ⓪   |     |
| `1` |     |     |  ⓪  |     |
| `2` |  ⓪   |  ⓪   |     |   ⓪  |
| `3` |     |     |   ⓪  |     |

Production:
    0 b → 0
    1 b -> 2
    3 b -> 2

0, 2 are not equivalent, so '0', '1' and '0,3' are not equivalent
|     | `0` | `1` | `2` | `3` |
|:---:|:---:|:---:|:---:|:---:|
| `0` |     |     |  ⓪   |     |
| `1` |     |     |  ⓪  |     |
| `2` |  ⓪   |  ⓪   |     |   ⓪  |
| `3` |     |     |   ⓪  |     |

The new states are {0}, {1, 3}, {2}.
The new initial state is the one that contains the old start state, i.e. `{0}`

The new final states are those that contain the old final state, i.e.. `{2}`

The new transitions:
    {0} a → {1,3}
    {0} b → {0}
    {1,3} a → {1,3}  
1 b → 2  
2 a → 3  
2 b → 2  
3 a → 3  
3 b → 2  

In [ ]:
class set(frozenset):
    def __repr__(self):
        return '{' + ', '.join(str(e) for e in self) + '}'

class FiniteStateAutomaton:
    def __init__(self, T, Q, R, q0, F):
        self.T, self.Q, self.R, self.q0, self.F = T, Q, R, q0, F
    def __repr__(self):
        return str(self.q0) + '\n' + ' '.join(str(f) for f in self.F) + '\n' + \
               '\n'.join(str(q) + ' ' + a + ' → ' + str(r) for (q, a, r) in self.R)

def parseFSA(fsa: str) -> FiniteStateAutomaton:
    fsa = [line for line in fsa.split('\n') if line.strip() != '']
    q0 = fsa[0] # first line: initial
    F = set(fsa[1].split()) # second line: final, final, ...
    R = set()
    for line in fsa[2:]: # all subsequent lines: "source symbol → target"
        l, r = line.split('→')
        R |= {(l.split()[0], l.split()[1], r.split()[0])}
    T = {r[1] for r in R}
    Q = {q0} | F | {r[0] for r in R} | {r[2] for r in R}
    return FiniteStateAutomaton(T, Q, R, q0, F)

def minimizeFSA(fsa: FiniteStateAutomaton) -> FiniteStateAutomaton:
    δ = {(q, a): r for (q, a, r) in fsa.R}
    dist = {(q, r) for q in fsa.Q for r in fsa.Q if q != r and (q in fsa.F) != (r in fsa.F)}
    done = False
    while not done:
        done = True #; print(dist)
        for q in fsa.Q:
            for r in fsa.Q:
                if q != r and (q, r) not in dist and any(((q, u) in δ) != ((r, u) in δ) or \
                    ((q, u) in δ) and ((δ[(q, u)], δ[(r, u)]) in dist) for u in fsa.T):
                    dist |= {(q, r)}; done = False #; print('adding', q, r)
    Qʹ = {set({q} | {r for r in fsa.Q if (q, r) not in dist}) for q in fsa.Q}
    Rʹ = {(qʹ, u, rʹ) for qʹ in Qʹ for rʹ in Qʹ for u in fsa.T if any((q, u, r) in fsa.R for q in qʹ for r in rʹ)}
    qʹ0 = {qʹ for qʹ in Qʹ if fsa.q0 in qʹ}.pop()
    Fʹ = {qʹ for qʹ in Qʹ if (qʹ & fsa.F) != set()}
    return FiniteStateAutomaton(fsa.T, Qʹ, Rʹ, qʹ0, Fʹ)

In [ ]:
Your code here